# Final Project: Foundation of Inference and Learning

#### Reproduce the results of  *Understanding Deep Learning Requires Rethinking Generalization*

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import matplotlib.pyplot as plt
from torchsummary import summary
import time
import random
import os
import warnings

from models import *
from utils import *

import ssl
ssl._create_default_https_context = ssl._create_unverified_context
warnings.filterwarnings("ignore")

output_dir = "saved_models"
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

### **Effective capacity of neural networks**

#### Reproduce Fig. 1(a)

In [ ]:
model = Inception_small()

# train the Inception with various modifications on CIFAR10
for corrupt_type in ['true labels', 'random label', 'shuffle pixels', 'random pixels', 'Gaussian']:
    # initialize model parameters
    random_seed(42)
    model.__init__() 
    # generate train set using specific corruption on labels or pixels
    cifar10_train = cifar_10_train(corrupt_type) 
    # train model
    trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True) 
    train(model, train_dataloader=trainloader, lr=0.1, max_epoch=65, save_dir='Fig_1a-{}'.format(corrupt_type))

#### Reproduce Fig. 2(b) and (c)

In [ ]:
# download test set and create test loader
transform = transforms.Compose([transforms.ToTensor(),
                 transforms.CenterCrop((28, 28)),
                 transforms.Lambda(lambda x: x / 255.0),
                 transforms.Lambda(per_image_standardization)])
cifar10_test = torchvision.datasets.CIFAR10(root='datasets', train=False, transform=transform, download=True)
testloader = torch.utils.data.DataLoader(cifar10_test, batch_size=128, shuffle=False) 

# Train the three models on CIFAR10, with various level of label corruption

model = Inception_small()
for label_corrpution_p in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    # initialize model parameters
    random_seed(42)
    model.__init__() 
    # generate train set using specific corruption on labels or pixels
    cifar10_train = cifar_10_train('partially corrupted labels', corrupt_p=label_corrpution_p) 
    # train model
    trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True) 
    train_val(model, train_dataloader=trainloader, test_loader = testloader, lr=0.1, max_epoch=2000, save_dir='Fig_1b-Inception_p{}'.format(str(label_corrpution_p)), early_stop=True)

model = Alexnet_small()
for label_corrpution_p in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    # initialize model parameters
    random_seed(42)
    model.__init__() 
    # generate train set using specific corruption on labels or pixels
    cifar10_train = cifar_10_train('partially corrupted labels', corrupt_p=label_corrpution_p) 
    # train model
    trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True) 
    train_val(model, train_dataloader=trainloader, test_loader = testloader, lr=0.01, max_epoch=2000, save_dir='Fig_1b-Alexnet_p{}'.format(str(label_corrpution_p)), early_stop=True)

model = MLP(layer=1)
for label_corrpution_p in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    # initialize model parameters
    random_seed(42)
    model.__init__() 
    # generate train set using specific corruption on labels or pixels
    cifar10_train = cifar_10_train('partially corrupted labels', corrupt_p=label_corrpution_p) 
    # train model
    trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True) 
    train_val(model, train_dataloader=trainloader, test_loader = testloader, lr=0.01, max_epoch=2000, save_dir='Fig_1b-MLP_p{}'.format(str(label_corrpution_p)), early_stop=True, min_delta=0, gamma=0.98)

#### Plot the final results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 

plt.figure(figsize=(15, 4))
plt.subplot(1, 3, 1)
nb_epochs = 65

corrupt_type_args = [{'corrupt type': 'true labels', 'color': 'blue', 'marker': 's'}, 
           {'corrupt type': 'random label', 'color': 'red', 'marker': 'o'},
           {'corrupt type': 'shuffle pixels', 'color': 'lawngreen', 'marker': '*'},
           {'corrupt type': 'random pixels', 'color': 'purple', 'marker': ''},
           {'corrupt type': 'Gaussian', 'color': 'black', 'marker': 'D'}]

for plt_info in corrupt_type_args:
    train_stats = np.load('saved_models/Fig_1a-{}-train_stats.npy'.format(plt_info['corrupt type']), allow_pickle=True)
    average_loss = [epoch_info['train_loss'] for epoch_info in train_stats]
    plt.plot(range(nb_epochs), average_loss, label=plt_info['corrupt type'], color=plt_info['color'], marker=plt_info['marker'], markevery=2, linewidth=2.5, markeredgecolor='black')
plt.legend(fontsize=15, loc='upper right')
plt.ylabel('average_loss', fontsize=15)
plt.xlabel('thousand steps', fontsize=15)
plt.xticks(ticks=np.array([0, 5, 10, 15, 20, 25])/0.391, labels=[0, 5, 10, 15, 20, 25])
plt.xlim(0, 25/0.391)
plt.ylim(0, 2.5)

# plt.figure(figsize=(12, 5))
label_corruption = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

corrupt_type_args = [{'model': 'Inception', 'color': 'black', 'marker': 's'}, 
           {'model': 'Alexnet', 'color': 'lawngreen', 'marker': 'o'},
           {'model': 'MLP', 'color': 'blue', 'marker': '*'}]

for plt_info in corrupt_type_args:
    model_name = 'Inception' if plt_info['model']=='Inception' else 'AlexNet' if plt_info['model']=='Alexnet' else 'MLP 1x512'
    train_time, test_error = [], []
    for corrupt_p in label_corruption:
        train_stats = np.load('saved_models/Fig_1b-{}_p{}-train_stats.npy'.format(plt_info['model'], str(corrupt_p)), allow_pickle=True)
        train_time.append(train_stats[-1]['train_time'])
        test_error.append(1-train_stats[-1]['test_acc'])
    train_time = np.array(train_time) / train_time[0]
    plt.subplot(1, 3, 2)
    plt.plot(label_corruption, train_time, label=model_name, color=plt_info['color'], marker=plt_info['marker'], linewidth=3, markeredgecolor='black')
    plt.subplot(1, 3, 3)
    plt.plot(label_corruption, test_error, label=model_name, color=plt_info['color'], marker=plt_info['marker'], linewidth=3, markeredgecolor='black')

plt.subplot(1, 3, 2)
plt.legend(fontsize=15, loc='upper left')
plt.ylabel('time to overfit', fontsize=15)
plt.xlabel('label corruption', fontsize=15)
plt.xticks(ticks=label_corruption[::2]*10, labels=label_corruption[::2])
plt.yticks(ticks=np.arange(1.0, 5.0, 0.5), labels=np.arange(1.0, 5.0, 0.5))
plt.xlim(0, 1)
plt.ylim(1, 4.0)

plt.subplot(1, 3, 3)
plt.plot(label_corruption, [0.9]*len(label_corruption), linestyle='--', color='red', linewidth=3)
plt.legend(fontsize=15, loc='bottom right')
plt.ylabel('test_error', fontsize=15)
plt.xlabel('label corruption', fontsize=15)
plt.xticks(ticks=label_corruption[::2]*10, labels=label_corruption[::2])
plt.xlim(0, 1)
plt.ylim(0.1, 1)

plt.savefig('fig_1.png', dpi=300)
plt.show()

### **Explicit regularizations**

In [ ]:
# load test set
transform = transforms.Compose([transforms.ToTensor(),
                 transforms.CenterCrop((28, 28)),
                 transforms.Lambda(lambda x: x / 255.0),
                 transforms.Lambda(per_image_standardization)])
cifar10_test = torchvision.datasets.CIFAR10(root='datasets', train=False, transform=transform, download=True)
testloader = torch.utils.data.DataLoader(cifar10_test, batch_size=128, shuffle=False)

In [ ]:
# train Inception model with batch normalization
model = Inception_small(use_bn=True)
for data_aug in ('True','False'):
    cifar10_train = cifar_10_train('true labels', data_augmentation=data_aug) 
    trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True)
    for wd in [0,5e-5]:
        random_seed(42)
        model.__init__()
        train_val(model, train_dataloader=trainloader, test_loader = testloader, lr=0.1, max_epoch=200, early_stop=True, min_delta=1e-5, weight_decay=wd)


In [ ]:
# train Inception model without batch normalization
model = Inception_small(use_bn=False)
cifar10_train = cifar_10_train('true labels', data_augmentation=False) 
trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True)
for wd in [0,5e-5]:
    random_seed(42)
    model.__init__()
    train_val(model, train_dataloader=trainloader, test_loader = testloader, lr=0.1, max_epoch=200, early_stop=True, min_delta=1e-5, weight_decay=wd)

In [ ]:
# train Alexnet model
model = Alexnet_small()
for data_aug in ('True','False'):
    cifar10_train = cifar_10_train('true labels', data_augmentation=data_aug) 
    trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True)
    for wd in [0,5e-5]:
        random_seed(42)
        model.__init__()
        train_val(model, train_dataloader=trainloader, test_loader = testloader, lr=0.01, max_epoch=200, early_stop=True, min_delta=1e-5, weight_decay=wd)

In [ ]:
# train MLP with three hidden layers
model = MLP(layer=3)
cifar10_train = cifar_10_train('true labels', data_augmentation=False) 
trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True)
for wd in [0,5e-5]:
    random_seed(42)
    model.__init__()
    train_val(model, train_dataloader=trainloader, test_loader = testloader, lr=0.01, max_epoch=200, early_stop=True, min_delta=1e-5, weight_decay=wd, gamma=0.98)

In [ ]:
# train MLP with one hidden layer
model = MLP(layer=1)
cifar10_train = cifar_10_train('true labels', data_augmentation=False) 
trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True)
for wd in [0,5e-5]:
    random_seed(42)
    model.__init__()
    train_val(model, train_dataloader=trainloader, test_loader = testloader, lr=0.01, max_epoch=200, early_stop=True, min_delta=1e-5, weight_decay=wd, gamma=0.98)

#### Reproduce Table1

In [ ]:
# load train data with random labels
cifar10_train = cifar_10_train('random label', data_augmentation=False) 
trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True)
model_incep_bn = Inception_small(use_bn=True)
model_incep = Inception_small(use_bn=False)
model_ale = Alexnet_small()
model_mlp3 = MLP(layer=3)
model_mlp1 = MLP(layer=1)

# set random seed
random_seed(42)

# conduct the experiments to reproduce table 1 in the paper
model_incep_bn.__init__()
train_val(model_incep_bn, train_dataloader=trainloader, test_loader = testloader, lr=0.1, max_epoch=200, early_stop=True, min_delta=1e-5)
model_incep.__init__()
train_val(model_incep, train_dataloader=trainloader, test_loader = testloader, lr=0.1, max_epoch=200, early_stop=True, min_delta=1e-5)
model_ale.__init__()
train_val(model_ale, train_dataloader=trainloader, test_loader = testloader, lr=0.01, max_epoch=200, early_stop=True, min_delta=1e-5)
model_mlp3.__init__()
train_val(model_mlp3, train_dataloader=trainloader, test_loader = testloader, lr=0.01, max_epoch=200, early_stop=True, min_delta=1e-5, gamma=0.98)
model_mlp1.__init__()
train_val(model_mlp1, train_dataloader=trainloader, test_loader = testloader, lr=0.01, max_epoch=200, early_stop=True, min_delta=1e-5, gamma=0.98)

### **Implicit regularizations**

#### Reproduce Fig. 2(a)

Train and Test Inception on Cifar-100 **without Data augmentation, Weight decay and Dropout**

In [ ]:
# set random seed
random_seed(seed=42)

In [ ]:
print('Inception with bn:')
model = Inception_small(num_classes=100, use_bn=True)
if torch.cuda.is_available():
    model.cuda()
print(summary(model, (3, 28, 28)))

In [ ]:
train_loader, test_loader = return_loaders(root='datasets', batch_size=128, db='cifar100', augmentation=False)

m1 = 'Length of iters per epoch: {}. Length of testing batches: {}.'
print(m1.format(len(train_loader), len(test_loader)))

In [ ]:
train_val(model, train_loader, test_loader, lr=0.1, max_epoch=250, save_dir='cifar100_without_aug_wd_dropout', predict_batch_bool=True)

Train and Test Inception on Cifar-100 **with Data augmentation, Weight decay and Dropout**

In [ ]:
# set random seed
random_seed(seed=42)

In [ ]:
print('Inception with bn and dropout:')
model = Inception_small(num_classes=100, use_bn=True, dropout=0.2)
if torch.cuda.is_available():
    model.cuda()
print(summary(model, (3, 28, 28)))

In [ ]:
train_loader, test_loader = return_loaders(root='datasets', batch_size=128, db='cifar100', augmentation=True)

m1 = 'Length of iters per epoch: {}. Length of testing batches: {}.'
print(m1.format(len(train_loader), len(test_loader)))

In [ ]:
train_val(model, train_loader, test_loader, lr=0.1, max_epoch=250, save_dir='cifar100_with_aug_wd_dropout', weight_decay=4e-5, predict_batch_bool=True)

Train and Test Inception on Cifar-100 **only with Weight decay**

---



In [ ]:
# set random seed
random_seed(seed=42)

In [ ]:
print('Inception with bn')
model = Inception_small(num_classes=100, use_bn=True)
if torch.cuda.is_available():
    model.cuda()
print(summary(model, (3, 28, 28)))

In [ ]:
train_loader, test_loader = return_loaders(root='datasets', batch_size=128, db='cifar100', augmentation=False)

m1 = 'Length of iters per epoch: {}. Length of testing batches: {}.'
print(m1.format(len(train_loader), len(test_loader)))

In [ ]:
train_val(model, train_loader, test_loader, lr=0.1, max_epoch=250, save_dir='cifar100_without_aug_dropout_with_wd', weight_decay=4e-5, predict_batch_bool=True)

**Plot training accuracy and testing accuracy**

In [ ]:
# load training states

cifar100_with_aug_wd_dropout_train_stats = np.load('saved_models/cifar100_with_aug_wd_dropout-train_stats.npy', allow_pickle=True)
cifar100_with_aug_wd_dropout_train_stats_batch = np.load('saved_models/cifar100_with_aug_wd_dropout-train_stats_batch.npy', allow_pickle=True)

cifar100_without_aug_dropout_with_wd_train_stats = np.load('saved_models/cifar100_without_aug_dropout_with_wd-train_stats.npy', allow_pickle=True)
cifar100_without_aug_dropout_with_wd_train_stats_batch = np.load('saved_models/cifar100_without_aug_dropout_with_wd-train_stats_batch.npy', allow_pickle=True)

cifar100_without_aug_wd_dropout_train_stats = np.load('saved_models/cifar100_without_aug_wd_dropout-train_stats.npy', allow_pickle=True)
cifar100_without_aug_wd_dropout_train_stats_batch = np.load('saved_models/cifar100_without_aug_wd_dropout-train_stats_batch.npy', allow_pickle=True)

In [ ]:
# load training accuracy and testing accuracy from training states
cifar100_with_aug_wd_dropout_train_acc, cifar100_with_aug_wd_dropout_train_acc_batch, cifar100_with_aug_wd_dropout_test_acc =  get_stats(cifar100_with_aug_wd_dropout_train_stats, cifar100_with_aug_wd_dropout_train_stats_batch, max_epoch=200)

cifar100_without_aug_dropout_with_wd_train_acc, cifar100_without_aug_dropout_with_wd_train_acc_batch, cifar100_without_aug_dropout_with_wd_test_acc =  get_stats(cifar100_without_aug_dropout_with_wd_train_stats, cifar100_without_aug_dropout_with_wd_train_stats_batch, max_epoch=200)
cifar100_without_aug_wd_dropout_train_acc, cifar100_without_aug_wd_dropout_train_acc_batch, cifar100_without_aug_wd_dropout_test_acc =  get_stats(cifar100_without_aug_wd_dropout_train_stats, cifar100_without_aug_wd_dropout_train_stats_batch, max_epoch=200)

In [ ]:
# compute the cumulative best test accuracy, as an indicator of potential performance gain of early stopping.
cifar100_with_aug_wd_dropout_test_acc_cul = culmulative_test_acc(cifar100_with_aug_wd_dropout_test_acc)
cifar100_without_aug_dropout_with_wd_test_acc_cul = culmulative_test_acc(cifar100_without_aug_dropout_with_wd_test_acc)
cifar100_without_aug_wd_dropout_test_acc_cul = culmulative_test_acc(cifar100_without_aug_wd_dropout_test_acc)

In [ ]:
# reproduce the fig2.a in the paper
Epochs = np.arange(200)
batches = np.arange(0,200*400, 100)

plt.figure(figsize=(10, 8))
plt.plot(Epochs*391/1000, cifar100_with_aug_wd_dropout_test_acc, marker='o', markevery=2, c='tab:blue', label='test(w/ aug, wd, dropout)')
plt.plot(batches/1000, cifar100_with_aug_wd_dropout_train_acc_batch,  marker='o',  markevery=2, c='tab:blue', alpha=0.5, label='train(w/ aug, wd, dropout)')
plt.fill_between(Epochs*391/1000, cifar100_with_aug_wd_dropout_test_acc_cul, cifar100_with_aug_wd_dropout_test_acc,
                alpha=0.5, edgecolor='tab:blue', facecolor='tab:blue')

plt.plot(Epochs*391/1000, cifar100_without_aug_dropout_with_wd_test_acc, marker='D', markevery=2, c='orangered', label='test(w/o aug, dropout)')
plt.plot(batches/1000, cifar100_without_aug_dropout_with_wd_train_acc_batch, marker='D', markevery=2, c='orangered', alpha=0.5, label='train(w/o aug, dropout)')
plt.fill_between(Epochs*391/1000, cifar100_without_aug_dropout_with_wd_test_acc_cul, cifar100_without_aug_dropout_with_wd_test_acc, 
                 alpha=0.5, edgecolor='orangered', facecolor='orangered')

plt.plot(Epochs*391/1000, cifar100_without_aug_wd_dropout_test_acc, c='yellow', label='test(w/o aug, wd, dropout)')
plt.plot(batches/1000, cifar100_without_aug_wd_dropout_train_acc_batch, c='yellow', alpha=0.5, label='train(w/o aug, wd, dropout)')
plt.fill_between(Epochs*391/1000, cifar100_without_aug_wd_dropout_test_acc_cul, cifar100_without_aug_wd_dropout_test_acc, 
                 alpha=0.5, edgecolor='yellow', facecolor='orangered')

y_ticks = np.arange(0, 1.05, 0.2)
plt.ylabel('accuracy', fontsize=15)
plt.xlabel('thousand training steps', fontsize=15)
plt.yticks(y_ticks, fontsize=15)
plt.grid()
plt.legend(fontsize=15)
plt.savefig('cifar100_plot.png', bbox_inches='tight')

#### Reproduce Fig. 2(b)

Train and Test Inception **with BN** on Cifar-10

In [ ]:
# set random seed
random_seed(seed=42)

In [ ]:
print('Inception with bn:')
Inception_with_bn = Inception_small(use_bn=True, init_scale=0.5)
if torch.cuda.is_available():
    Inception_with_bn.cuda()
print(summary(Inception_with_bn, (3, 28, 28)))

In [ ]:
train_loader, test_loader = return_loaders(root='datasets', batch_size=128, db='cifar', augmentation=False)

m1 = 'Length of iters per epoch: {}. Length of testing batches: {}.'
print(m1.format(len(train_loader), len(test_loader)))

In [ ]:
train_val(Inception_with_bn, train_loader, test_loader, lr=0.1, max_epoch=100, save_dir='Inception_with_bn', predict_batch_bool=True)

Train and Test Inception **without BN** on Cifar-10

In [ ]:
# set random seed
random_seed(seed=42)

In [ ]:
print('Inception without bn:')
Inception_without_bn = Inception_small(use_bn=False, init_scale=0.5)
if torch.cuda.is_available():
    Inception_without_bn.cuda()
print(summary(Inception_without_bn, (3, 28, 28)))

In [ ]:
train_loader, test_loader = return_loaders(root='datasets', batch_size=128, db='cifar', augmentation=False)

m1 = 'Length of iters per epoch: {}. Length of testing batches: {}.'
print(m1.format(len(train_loader), len(test_loader)))

In [ ]:
train_val(Inception_without_bn, train_loader, test_loader, lr=0.1, max_epoch=100, save_dir='Inception_without_bn', predict_batch_bool=True)

**Plot training accuracy and testing accuracy**

In [ ]:
# load training states
Inception_with_bn_train_stats = np.load('saved_models/Inception_with_bn-train_stats.npy', allow_pickle=True)
Inception_with_bn_train_stats_batch = np.load('saved_models/Inception_with_bn-train_stats_batch.npy', allow_pickle=True)


Inception_without_bn_train_stats = np.load('saved_models/Inception_without_bn-train_stats.npy', allow_pickle=True)
Inception_without_bn_train_stats_batch = np.load('saved_models/Inception_without_bn-train_stats_batch.npy', allow_pickle=True)

Inception_with_bn_train_acc, Inception_with_bn_train_acc_batch, Inception_with_bn_test_acc =  get_stats(Inception_with_bn_train_stats, Inception_with_bn_train_stats_batch, max_epoch=52)
Inception_without_bn_train_acc, Inception_without_bn_train_acc_batch, Inception_without_bn_test_acc =  get_stats(Inception_without_bn_train_stats, Inception_without_bn_train_stats_batch, max_epoch=52)



In [ ]:
# compute the cumulative best test accuracy, as an indicator of potential performance gain of early stopping.
Inception_with_bn_test_acc_cul = culmulative_test_acc(Inception_with_bn_test_acc)
Inception_without_bn_test_acc_cul = culmulative_test_acc(Inception_without_bn_test_acc)

In [ ]:
# reproduce fig2.b in the paper
Epochs = np.arange(52)
batches = np.arange(0, 52*400, 100)

plt.figure(figsize=(10, 8))
plt.plot(Epochs*391/1000, Inception_with_bn_test_acc, marker='o', c='tab:blue', label='test(Inception)')
plt.plot(batches/1000, Inception_with_bn_train_acc_batch,  marker='o',  c='tab:blue', alpha=0.3, label='train(Inception)')
plt.fill_between(Epochs*391/1000, Inception_with_bn_test_acc_cul, Inception_with_bn_test_acc, 
                 Inception_with_bn_test_acc, alpha=0.5, edgecolor='tab:blue', facecolor='tab:blue')

plt.plot(Epochs*391/1000, Inception_without_bn_test_acc, c='orangered', label='test(Inception w/o BN)')
plt.plot(batches/1000, Inception_without_bn_train_acc_batch, c='orangered', alpha=0.3, label='train(Inception w/o BN)')
plt.fill_between(Epochs*391/1000, Inception_without_bn_test_acc_cul, Inception_without_bn_test_acc, 
                 Inception_without_bn_test_acc, alpha=0.5, edgecolor='orangered', facecolor='orangered')

x_ticks = np.arange(0, 25, 5)
y_ticks = np.arange(0.6, 1.05, 0.1)
plt.ylabel('accuracy', fontsize=15)
plt.xlabel('thousand training steps', fontsize=15)
plt.ylim(0.6, 1.0)
plt.xticks(x_ticks, fontsize=15)
plt.yticks(y_ticks, fontsize=15)
plt.grid()
plt.legend(fontsize=15, loc='lower right')
plt.savefig('cifar10_plot.png', bbox_inches='tight')


**Train and test Inception with BN and 64 batch size**


In [ ]:
# set random seed
random_seed(seed=42)

In [ ]:
print('Inception with bn:')
Inception_with_bn = Inception_small(use_bn=True, init_scale=0.5)
if torch.cuda.is_available():
    Inception_with_bn.cuda()
print(summary(Inception_with_bn, (3, 28, 28)))

In [ ]:
train_loader, test_loader = return_loaders(root='datasets', batch_size=64, db='cifar', augmentation=False)

m1 = 'Length of iters per epoch: {}. Length of testing batches: {}.'
print(m1.format(len(train_loader), len(test_loader)))

In [ ]:
train_val(Inception_with_bn, train_loader, test_loader, lr=0.1, max_epoch=100, save_dir='Inception_with_bn_batch_size_64', weight_decay=0, record_batch=200, predict_batch_bool=True)

Plot training accuracy and testing accuracy for Inception **with BN and different batch sizes**

In [ ]:
# load training states of Inception with BN and 128 batch size
Inception_with_bn_train_stats = np.load('saved_models/Inception_with_bn-train_stats.npy', allow_pickle=True) 
Inception_with_bn_train_stats_batch = np.load('saved_models/Inception_with_bn-train_stats_batch.npy', allow_pickle=True)

# load training states of Inception with BN and 64 batch size
Inception_with_bn_64_train_stats = np.load('saved_models/Inception_with_bn_batch_size_64-train_stats.npy', allow_pickle=True)
Inception_with_bn_64_train_stats_batch = np.load('saved_models/Inception_with_bn_batch_size_64-train_stats_batch.npy', allow_pickle=True)

# load training accuracy and test accuracy from the training states
Inception_with_bn_train_acc, Inception_with_bn_train_acc_batch, Inception_with_bn_test_acc =  get_stats(Inception_with_bn_train_stats, Inception_with_bn_train_stats_batch, max_epoch=100)
Inception_with_bn_64_train_acc, Inception_with_bn_64_train_acc_batch, Inception_with_bn_64_test_acc =  get_stats(Inception_with_bn_64_train_stats, Inception_with_bn_64_train_stats_batch, max_epoch=52)

# compute the cumulative best test accuracy, as an indicator of potential performance gain of early stopping.
Inception_with_bn_test_acc_cul = culmulative_test_acc(Inception_with_bn_test_acc)
Inception_with_bn_64_test_acc_cul = culmulative_test_acc(Inception_with_bn_64_test_acc)

# plot the training accuracy and testing accuracy of Inception with BN and different batch sizes
Epochs_64 = np.arange(52)
Epochs_128 = np.arange(100)
batches_64 = np.arange(0, 52*800, 200)
batches_128 = np.arange(0, 100*400, 100)

plt.figure(figsize=(10, 8))
plt.plot(Epochs_128*391/1000, Inception_with_bn_test_acc, marker='o', markevery=2, c='tab:blue', label='test(Inception, batch size = 128)')
plt.plot(batches_128/1000, Inception_with_bn_train_acc_batch,  marker='o', markevery=2, c='tab:blue', alpha=0.3, label='train(Inception, batch size = 128)')
plt.fill_between(Epochs_128*391/1000, Inception_with_bn_test_acc_cul, Inception_with_bn_test_acc, 
                 Inception_with_bn_test_acc, alpha=0.5, edgecolor='tab:blue', facecolor='tab:blue')

plt.plot(Epochs_64*782/1000, Inception_with_bn_64_test_acc, c='g', label='test(Inception, batch size = 64)')
plt.plot(batches_64/1000, Inception_with_bn_64_train_acc_batch, c='g', alpha=0.3, label='train(Inception, batch size = 64)')
plt.fill_between(Epochs_64*782/1000, Inception_with_bn_64_test_acc_cul, Inception_with_bn_64_test_acc, 
                 Inception_with_bn_64_test_acc, alpha=0.5, edgecolor='g', facecolor='g')

x_ticks = np.arange(0, 45, 5)
y_ticks = np.arange(0.6, 1.05, 0.1)
plt.ylabel('accuracy', fontsize=15)
plt.xlabel('thousand training steps', fontsize=15)
plt.ylim(0.6, 1.0)
plt.xticks(x_ticks, fontsize=15)
plt.yticks(y_ticks, fontsize=15)
plt.grid()
plt.legend(fontsize=15, loc='lower right')
plt.savefig('cifar10_different_batch_size.png', bbox_inches='tight')